In [1]:
!pip install nltk==3.8.1 sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.5 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4f81cf1e9ac9967fd5fc07e5b8fa5b478d0ce37bcf1b244d86bc4380c627939e
  Stored in directory: /root/.cache/pip/wheels/83/71/2b/40d17d21937fed496fb99145227eca8f20b4891240ff60c86f
Successfully built sentence-transformers
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import BatchNormalization
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from tensorflow.keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import sequence, text
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Input, Embedding, Dropout, Conv1D
from tensorflow.keras.models import Model
from tensorflow.keras.losses import BinaryCrossentropy
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
from sklearn.metrics import accuracy_score


In [3]:
df = pd.read_csv('/kaggle/input/ru-ukr-onlyv4/ru-ukr-onlyv4.csv')

df.loc[df['truth_value'] == 'tom_ruling_pof', 'truth_value'] = 'meter-false'
df.loc[df['truth_value'] == 'meter-half-true', 'truth_value'] = 'meter-true'
df.loc[df['truth_value'] == 'meter-mostly-true', 'truth_value'] = 'meter-true'
df.loc[df['truth_value'] == 'meter-mostly-false', 'truth_value'] = 'meter-false'
df = df.dropna(subset=['claim'])
df.reset_index(drop=True, inplace=True)
df['truth_value'].value_counts()

meter-true     1358
meter-false    1325
Name: truth_value, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
cdf = df.copy()
cdf['truth_value'] = le.fit_transform(cdf['truth_value'])

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
def stemm_text(text):
    return ' '.join([stemmer.stem(w) for w in text.split(' ')])

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in text.split(' ')])

T = cdf['claim'].str.split(' \n\n---\n\n').str[0]
T = T.str.replace('-',' ').str.replace('[^\w\s]','').str.replace('\n',' ').str.lower()
stop = stopwords.words('english')
T = T.apply(lambda x: ' '.join([y for y in x.split() if not y.isdigit()]))
T = T.apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))
cdf['claim'] = T
cdf.head(10)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  


,Unnamed: 0,claim,truth_value
0,0,provocation disinformation overview,0
1,1,ukraine theft homicide levels rose due power o...,0
2,2,ukrainians beat two berlin residents speaking ...,0
3,3,quote paul goebbels banderites,0
4,4,culture good neighborliness course ukrainian s...,0
5,5,us research ukraine led increase incidence tic...,0
6,6,chile law rights mutants genetically modified ...,0
7,7,covid incidence rate became zero late may,0
8,8,risk death among children vaccinated covid tim...,0
9,9,provocation disinformation overview,0


In [6]:
X = cdf['claim']
y = cdf['truth_value']

X = X.apply(lambda w: lemmatize_text(w))
X

0                     provocation disinformation overview
1       ukraine theft homicide level rose due power ou...
2       ukrainian beat two berlin resident speaking ru...
3                          quote paul goebbels banderites
4       culture good neighborliness course ukrainian s...
                              ...                        
2678    global ballet community rally help young ukrai...
2679    majority american say russia enemy differ hand...
2680    china indicated le interest discussing violenc...
2681    sanction punishing russia economy china econom...
2682    thousand mile away ukraine russian invasion pl...
Name: claim, Length: 2683, dtype: object

In [7]:
torch.manual_seed(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')

In [8]:
def tokenizeAndGenerateSequences(X, y):
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=42)

    #Maximum number of characters in a sentence = 44
    tk = text.Tokenizer(num_words=1000)
    tk.fit_on_texts(xtrain)
    tokenized_train = tk.texts_to_sequences(xtrain)
    X_train = torch.tensor(sequence.pad_sequences(tokenized_train, maxlen=60)).to(device)
    tokenized_test = tk.texts_to_sequences(xtest)
    X_test = torch.tensor(sequence.pad_sequences(tokenized_test, maxlen=60)).to(device)

    # Convert labels to tensors
    y_train = torch.tensor(ytrain.values).float().to(device)
    y_test = torch.tensor(ytest.values).float().to(device)
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = tokenizeAndGenerateSequences(cdf['claim'], cdf['truth_value'])
X_train, y_train, X_test, y_test = tokenizeAndGenerateSequences(X, y)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([2146, 60]),
 torch.Size([2146]),
 torch.Size([537, 60]),
 torch.Size([537]))

In [ ]:

# Define the model architecture
class BiLSTM(nn.Module):
    def __init__(self, num_words, embed_size, hidden_size, output_size, dropout_rate):
        super(BiLSTM, self).__init__()
        self.embedding = nn.Embedding(num_words, embed_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.bilstm1 = nn.LSTM(embed_size, hidden_size, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(hidden_size * 2, hidden_size, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x, _ = self.bilstm1(x)
        x, _ = self.bilstm2(x)
        x = self.fc(x[:, -1, :])
        x = self.sigmoid(x)
        return x
    

# Train the model
te = 50
acc = []
tracc = []
for e in range(1, te+1):
    ctracc = 0
    model = BiLSTM(num_words=1000, embed_size=60, hidden_size=64, output_size=1, dropout_rate=0.2).to(device)
    criterion = nn.BCELoss().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.01)

    model.train()
    for epoch in range(e):
        optimizer.zero_grad()
        outputs = model(X_train.to(device))
        loss = criterion(outputs.squeeze(), y_train)
        loss.backward()
        optimizer.step()

        with torch.no_grad():
            predictions = model(X_train.to(device))
            predictions = (predictions > 0.5).to('cpu').int().squeeze().numpy()
        train_accuracy = metrics.accuracy_score(y_train.to('cpu'), predictions)
        ctracc += train_accuracy

    ctracc /= e

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        predictions = model(X_test)
        predictions = (predictions > 0.5).to('cpu').int().squeeze().numpy()

    print(f"Total Epochs: {e}, Train Accuracy: {ctracc} Test Accuracy: {metrics.accuracy_score(y_test.to('cpu'), predictions)}")
    acc.append(metrics.accuracy_score(y_test.to('cpu'), predictions))
    tracc.append(ctracc)
#     break
print('Max acc -', max(acc), ' with epochs -', acc.index(max(acc)))
# plt.plot([i for i in range(1, 51)], acc)

Total Epochs: 1, Train Accuracy: 0.6766076421248836 Test Accuracy: 0.6629422718808193
Total Epochs: 2, Train Accuracy: 0.6593662628145387 Test Accuracy: 0.6256983240223464
Total Epochs: 3, Train Accuracy: 0.6545511028269648 Test Accuracy: 0.6405959031657356
Total Epochs: 4, Train Accuracy: 0.6826654240447344 Test Accuracy: 0.6964618249534451
Total Epochs: 5, Train Accuracy: 0.68928238583411 Test Accuracy: 0.6834264432029795
Total Epochs: 6, Train Accuracy: 0.6915191053122087 Test Accuracy: 0.7225325884543762
Total Epochs: 7, Train Accuracy: 0.7224071362002397 Test Accuracy: 0.7802607076350093
Total Epochs: 8, Train Accuracy: 0.7347972972972974 Test Accuracy: 0.776536312849162
Total Epochs: 9, Train Accuracy: 0.7580511546028788 Test Accuracy: 0.7746741154562383
Total Epochs: 10, Train Accuracy: 0.7870456663560111 Test Accuracy: 0.819366852886406
Total Epochs: 11, Train Accuracy: 0.7654833516902482 Test Accuracy: 0.7914338919925512
Total Epochs: 12, Train Accuracy: 0.766192917054986 Test